<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/c3_w2_embeddings_hyperparameters_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings hyperparameters tunning

In [3]:
!wget \
  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json

--2020-11-15 06:14:56--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2020-11-15 06:14:56 (159 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [1]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
vocab_size = 10000
embedding_dim = 16
max_lenght = 100
trunc_type='post'
padding_type = 'post'
oov_tok = '<oov>'
training_size = 20000

In [8]:
sentences = []
labels = []

with open('sarcasm.json', 'r') as f:
  datastore = json.load(f)
  for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

len(sentences), len(labels)

(26709, 26709)

In [11]:
training_sentences = sentences[:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[:training_size]
testing_labels = labels[training_size:]

len(training_sentences), len(testing_sentences), \
len(training_labels), len(testing_labels)

(20000, 6709, 20000, 6709)

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokernizer.word_index

training_sequences = tokenizer.tests_to_sequences(training_sentences)
training_padded = pad_sequences(
    training_sequences, 
    maxlen=max_lenght, 
    padding=padding_type,
    truncating=trunc_type
)

testing_sequences = tokenizer.tests_to_sequences(testing_sentences)
testing_padded = pad_sequences(
    testing_sequences, 
    maxlen=max_lenght, 
    padding=padding_type,
    truncating=trunc_type
)



testing_sentences